In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# import os
# os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async" # this is highly important for totally utilizing your GPU's memory, but it also breaks the profiler's memory breakdown

import numpy as np
import qutip as qt 
from QOGS.optimizer.tf_adam_optimizer import AdamOptimizer
from QOGS.gate_sets import SNAP_disp
from QOGS.optimizer.GateSynthesizer import GateSynthesizer
import matplotlib.pyplot as plt

In [ ]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
N = 40
# We initialize the ECDGateSet here
gate_set_params = {
    'N_cav' : N,
    'N_snap' : 10,
    'beta_scale' : 3.0
}
SNAP_gate_set = SNAP_disp(**gate_set_params)

In [3]:
#The target oscillator state.
X_gate = np.eye(N)
x = 1 / np.sqrt(2)
X_gate[0, 2] = X_gate[2, 0] = X_gate[2, 4] = X_gate[4, 2] = x
X_gate[0, 0] = X_gate[2, 2] = X_gate[4, 4] = 0
X_gate = qt.Qobj(X_gate)

In [4]:
X_gate.dag() * X_gate

Quantum object: dims = [[40], [40]], shape = (40, 40), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  ... 0.  0.  0. ]
 [0.  1.  0.  ... 0.  0.  0. ]
 [0.  0.  1.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 1.  0.  0. ]
 [0.  0.  0.  ... 0.  1.  0. ]
 [0.  0.  0.  ... 0.  0.  1. ]]

In [ ]:
#the optimization options
synth_params = {
    'gateset' : SNAP_gate_set,
    'N_blocks':3,
    'N_multistart' : 10000, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 100, #number of epochs before termination
    'epoch_size' : 10, #number of adam steps per epoch
    'learning_rate' : 0.01, #adam learning rate
    'term_fid' : 0.995, #terminal fidelitiy
    'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
    'beta_scale' : 3.0, #maximum |beta| for random initialization
    'initial_states' : [qt.basis(N,0)], #qubit tensor oscillator, start in |g> |0>
    'target_states' : [psi_t], #end in |e> |target>.
    'name' : 'SNAP Fock %d' % fock, #name for printing and saving
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}
gatesynth = GateSynthesizer(**synth_params)

In [ ]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(gatesynth)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
gatesynth.print_info()

In [ ]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
opt.optimize(logdir='logs')

In [ ]:
#can print info, including the best circuit found.
gatesynth.print_info() 

In [ ]:
print(gatesynth.best_fidelity())

In [ ]:
#can also get the best circuit parameters directly, could save this to a .npz file.
best_circuit =  gatesynth.best_circuit()
betas = best_circuit['betas']
thetas = best_circuit['thetas']
print(best_circuit)

In [3]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs

Reusing TensorBoard on port 6006 (pid 6367), started 0:10:19 ago. (Use '!kill 6367' to kill it.)